In [1]:
from pathlib import Path
import pandas as pd
import os

# Use pathlib to get the path of the current script
script_dir = os.getcwd()
print(script_dir)

/Users/nilsjennissen/PycharmProjects/directory-assistant/notebooks


In [2]:
class DisplayablePath(object):
    display_filename_prefix_middle = '├──'
    display_filename_prefix_last = '└──'
    display_parent_prefix_middle = '    '
    display_parent_prefix_last = '│   '

    def __init__(self, path, parent_path, is_last):
        self.path = Path(str(path))
        self.parent = parent_path
        self.is_last = is_last
        if self.parent:
            self.depth = self.parent.depth + 1
        else:
            self.depth = 0

    @property
    def displayname(self):
        if self.path.is_dir():
            return self.path.name + '/'
        return self.path.name

    @classmethod
    def make_tree(cls, root, parent=None, is_last=False, criteria=None):
        root = Path(str(root))
        criteria = criteria or cls._default_criteria

        displayable_root = cls(root, parent, is_last)
        yield displayable_root

        # Specify criteria to exclude directories/files
        children = sorted(list(path
                               for path in root.glob('*')  # for path in root.iterdir() <- all files
                               if not path.name.startswith('.') and criteria(path)),  # if criteria(path)),
                          key=lambda s: str(s).lower())
        count = 1
        for path in children:
            is_last = count == len(children)
            if path.is_dir():
                yield from cls.make_tree(path,
                                         parent=displayable_root,
                                         is_last=is_last,
                                         criteria=criteria)
            else:
                yield cls(path, displayable_root, is_last)
            count += 1

    @classmethod
    def _default_criteria(cls, path):
        return True

    @property
    def displayname(self):
        if self.path.is_dir():
            return self.path.name + '/'
        return self.path.name

    def displayable(self):
        if self.parent is None:
            return self.displayname

        _filename_prefix = (self.display_filename_prefix_last
                            if self.is_last
                            else self.display_filename_prefix_middle)

        parts = ['{!s} {!s}'.format(_filename_prefix,
                                    self.displayname)]

        parent = self.parent
        while parent and parent.parent is not None:
            parts.append(self.display_parent_prefix_middle
                         if parent.is_last
                         else self.display_parent_prefix_last)
            parent = parent.parent

        return ''.join(reversed(parts))


def get_tree_df(path, criteria=None):
    paths = list(DisplayablePath.make_tree(Path.home() / path, criteria=criteria))
    tree_list = []
    for path in paths:
        depth = path.depth
        name = path.displayname
        path_str = str(path.path)
        is_dir = path.path.is_dir()
        tree_list.append((depth, name, path_str, is_dir))
    df = pd.DataFrame(tree_list, columns=['depth', 'name', 'path', 'is_dir'])
    return df

In [3]:
path = "PycharmProjects/simulations"
dir_tree = get_tree_df(path)

print(dir_tree)

    depth                    name   
0       0            simulations/  \
1       1          conway_life.py   
2       1                   data/   
3       1                   gifs/   
4       2                7fpp.gif   
5       2                7fps.gif   
6       2                7fpw.gif   
7       2           animation.gif   
8       2      animation_test.gif   
9       2          extra_wide.gif   
10      2          gen_100_20.gif   
11      2            gen_1_40.gif   
12      2           gen_50_20.gif   
13      2           gen_50_40.gif   
14      2            gen_5_40.gif   
15      2                wide.gif   
16      2         wrong_order.gif   
17      1                 LICENSE   
18      1                 models/   
19      1              notebooks/   
20      2        create_gif.ipynb   
21      2  simulation_class.ipynb   
22      2   simulation_func.ipynb   
23      2     simulation_nn.ipynb   
24      1               README.md   
25      1        requirements.txt   
2

In [4]:
dir_tree.to_csv(f'../data/directory.csv', index=False)


## Enhancing the information

In [12]:
# Read CSV
dir = pd.read_csv('../data/directory.csv')

In [13]:
dir.head()

,depth,name,path,is_dir
0,0,simulations/,/Users/nilsjennissen/PycharmProjects/simulations,True
1,1,conway_life.py,/Users/nilsjennissen/PycharmProjects/simulatio...,False
2,1,data/,/Users/nilsjennissen/PycharmProjects/simulatio...,True
3,1,gifs/,/Users/nilsjennissen/PycharmProjects/simulatio...,True
4,2,7fpp.gif,/Users/nilsjennissen/PycharmProjects/simulatio...,False


,depth,name,path,is_dir,parent_folder
0,0,simulations/,/Users/nilsjennissen/PycharmProjects/simulations,True,PycharmProjects
1,1,conway_life.py,/Users/nilsjennissen/PycharmProjects/simulatio...,False,simulations
2,1,data/,/Users/nilsjennissen/PycharmProjects/simulatio...,True,simulations
3,1,gifs/,/Users/nilsjennissen/PycharmProjects/simulatio...,True,simulations
4,2,7fpp.gif,/Users/nilsjennissen/PycharmProjects/simulatio...,False,gifs
